In [ ]:
from collections import defaultdict
import pandas as pd
from datetime import datetime, date
loc_train = "D:\\Stat Team Project\\trainHistory.csv"
loc_test = "D:\\Stat Team Project\\testHistory.csv"
loc_offers = "D:\\Stat Team Project\\offers.csv"
loc_transactions = "D:\\Stat Team Project\\reducedDS2.csv"

In [ ]:
def diff_days(s1,s2):
    date_format = "%m/%d/%Y"
    date_format2 = "%Y-%m-%d"
    a = datetime.strptime(s1, date_format)
    b = datetime.strptime(s2, date_format2)
    delta = b - a
    return delta.days

In [ ]:
#keep a dictionary with the offerdata
offers = {}
for e, line in enumerate( open(loc_offers) ):
    row = line.strip().split(",")
    offers[ row[0] ] = row

#keep two dictionaries with the shopper id's from test and train
train_ids = {}
test_ids = {}
for e, line in enumerate( open(loc_train) ):
    if e > 0:
        row = line.strip().split(",")
        train_ids[row[0]] = row
for e, line in enumerate( open(loc_test) ):
    if e > 0:
        row = line.strip().split(",")
        test_ids[row[0]] = row

In [ ]:

#iterate through reduced dataset 
last_id = 0
features = defaultdict(float)
testdf = pd.DataFrame()
traindf = pd.DataFrame()
for e, line in enumerate( open(loc_transactions) ):
    if e > 0: #skip header
        #poor man's csv reader
        row = line.strip().split(",")
        #write away the features when we get to a new shopper id
        if last_id != row[0] and e != 1:

            #generate negative features
            if "has_bought_company" not in features:
                features['never_bought_company'] = 1

            if "has_bought_category" not in features:
                features['never_bought_category'] = 1

            if "has_bought_brand" not in features:
                features['never_bought_brand'] = 1

            if "has_bought_brand" in features and "has_bought_category" in features and "has_bought_company" in features:
                features['has_bought_brand_company_category'] = 1

            if "has_bought_brand" in features and "has_bought_category" in features:
                features['has_bought_brand_category'] = 1

            if "has_bought_brand" in features and "has_bought_company" in features:
                features['has_bought_brand_company'] = 1

            if features['label'] == 0.5:
                testdf = testdf.append(features, ignore_index=True)
            else:
                traindf = traindf.append(features, ignore_index=True)

            features = defaultdict(float)
        #generate features from transaction record
        #check if we have a test sample or train sample
        if row[0] in train_ids or row[0] in test_ids:

            #generate label and history
            if row[0] in train_ids:
                history = train_ids[row[0]]
                #print(history)
                if train_ids[row[0]][5] == "t":
                    features['label'] = 1
                else:
                    features['label'] = 0
            else:
                history = test_ids[row[0]]
                features['label'] = 0.5

            features['ID'] = row[0]
            features['offer_value'] = offers[ history[2] ][4]
            features['offer_quantity'] = offers[ history[2] ][2]
            offervalue = offers[ history[2] ][4]
            features['total_spend'] += float( row[10] )
    
            if offers[ history[2] ][3] == row[4]:
                features['has_bought_company'] += 1.0
                features['has_bought_company_q'] += float( row[9] )
                features['has_bought_company_a'] += float( row[10] )

                date_diff_days = diff_days(row[6],history[-1])
                if date_diff_days < 30:
                    features['has_bought_company_30'] += 1.0
                    features['has_bought_company_q_30'] += float( row[9] )
                    features['has_bought_company_a_30'] += float( row[10] )
                if date_diff_days < 60:
                    features['has_bought_company_60'] += 1.0
                    features['has_bought_company_q_60'] += float( row[9] )
                    features['has_bought_company_a_60'] += float( row[10] )
                if date_diff_days < 90:
                    features['has_bought_company_90'] += 1.0
                    features['has_bought_company_q_90'] += float( row[9] )
                    features['has_bought_company_a_90'] += float( row[10] )
                if date_diff_days < 180:
                    features['has_bought_company_180'] += 1.0
                    features['has_bought_company_q_180'] += float( row[9] )
                    features['has_bought_company_a_180'] += float( row[10] )
            if offers[ history[2] ][1] == row[3]:
                features['has_bought_category'] += 1.0
                features['has_bought_category_q'] += float( row[9] )
                features['has_bought_category_a'] += float( row[10] )
                date_diff_days = diff_days(row[6],history[-1])
                if date_diff_days < 30:
                    features['has_bought_category_30'] += 1.0
                    features['has_bought_category_q_30'] += float( row[9] )
                    features['has_bought_category_a_30'] += float( row[10] )
                if date_diff_days < 60:
                    features['has_bought_category_60'] += 1.0
                    features['has_bought_category_q_60'] += float( row[9] )
                    features['has_bought_category_a_60'] += float( row[10] )
                if date_diff_days < 90:
                    features['has_bought_category_90'] += 1.0
                    features['has_bought_category_q_90'] += float( row[9] )
                    features['has_bought_category_a_90'] += float( row[10] )						
                if date_diff_days < 180:
                    features['has_bought_category_180'] += 1.0
                    features['has_bought_category_q_180'] += float( row[9] )
                    features['has_bought_category_a_180'] += float( row[10] )				
            if offers[ history[2] ][5] == row[5]:
                features['has_bought_brand'] += 1.0
                features['has_bought_brand_q'] += float( row[9] )
                features['has_bought_brand_a'] += float( row[10] )
                date_diff_days = diff_days(row[6],history[-1])
                if date_diff_days < 30:
                    features['has_bought_brand_30'] += 1.0
                    features['has_bought_brand_q_30'] += float( row[9] )
                    features['has_bought_brand_a_30'] += float( row[10] )
                if date_diff_days < 60:
                    features['has_bought_brand_60'] += 1.0
                    features['has_bought_brand_q_60'] += float( row[9] )
                    features['has_bought_brand_a_60'] += float( row[10] )
                if date_diff_days < 90:
                    features['has_bought_brand_90'] += 1.0
                    features['has_bought_brand_q_90'] += float( row[9] )
                    features['has_bought_brand_a_90'] += float( row[10] )						
                if date_diff_days < 180:
                    features['has_bought_brand_180'] += 1.0
                    features['has_bought_brand_q_180'] += float( row[9] )
                    features['has_bought_brand_a_180'] += float( row[10] )	
        last_id = row[0]
        if e % 100000 == 0:
            print(e)


In [ ]:
testdf.to_csv("D:\\Stat Team Project\\testdf.csv")
traindf.to_csv("D:\\Stat Team Project\\traindf.csv")